In [1]:

import pandas as pd 
from fredapi import Fred
import missingno
import matplotlib.pyplot as plt 
plt.style.use('ggplot')


In [9]:
APIKEY = "3350621adf83e98b7ae0cf60ee5098a1"

fred = Fred(api_key=APIKEY)

series_codes = ["USREC", # US Recession
           "GDPC1", "PIECTR", "PRS85006013", "IPB50001SQ", "CQRMTSPL", # Macro
           "T10Y3M", "T10Y2Y", ## Term Spreads
           "BAMLH0A0HYM2", "BAA10Y", "AAAFF",  ## Corporate spreads
           "T10YIE", "T5YIE", ## Inflation spreads
           "VIXCLS", "GVZCLS","OVXCLS", # Volatility Index
           "ATLSBUSRGEP","ATLSBUEGEP",# Business Expectations
           "ATLSBUSRGUP","ATLSBUEGUP",# Business Uncertainty
           ]

data = []
series_infos = []
for serie in series_codes:
    try:
        info = pd.DataFrame(fred.get_series_info(serie)).T
        series_infos.append(info)
        df = fred.get_series(serie)
        df = df.to_frame()
        df.index = pd.to_datetime(df.index)
        df.columns= [serie]
        data.append(df)
    except Exception as e:
        print(e)
        print(f"Couldn't get {serie}")

In [13]:
infos = pd.concat(series_infos)

In [14]:
infos

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,USREC,2023-10-08,2023-10-08,NBER based Recession Indicators for the United...,1854-12-01,2023-09-01,Monthly,M,+1 or 0,+1 or 0,Not Seasonally Adjusted,NSA,2023-10-02 18:01:01-05,69,This time series is an interpretation of US Bu...
0,GDPC1,2023-10-08,2023-10-08,Real Gross Domestic Product,1947-01-01,2023-04-01,Quarterly,Q,Billions of Chained 2017 Dollars,Bil. of Chn. 2017 $,Seasonally Adjusted Annual Rate,SAAR,2023-09-28 07:53:02-05,91,BEA Account Code: A191RX Real gross domestic ...
0,PIECTR,2023-10-08,2023-10-08,Real personal income excluding current transfe...,1947-01-01,2023-04-01,Quarterly,Q,Billions of Chained 2017 Dollars,Bil. of Chn. 2017 $,Seasonally Adjusted Annual Rate,SAAR,2023-09-28 07:51:20-05,10,BEA Account Code: W875RX The current-dollar m...
0,PRS85006013,2023-10-08,2023-10-08,Nonfarm Business Sector: Employment for All Wo...,1947-01-01,2023-04-01,Quarterly,Q,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2023-09-07 07:36:28-05,3,NaN
0,IPB50001SQ,2023-10-08,2023-10-08,Industrial Production: Total Index,1919-01-01,2023-04-01,Quarterly,Q,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2023-09-15 08:28:02-05,15,The industrial production (IP) index measures ...
0,CQRMTSPL,2023-10-08,2023-10-08,Real Manufacturing and Trade Industries Sales,1967-01-01,2023-04-01,Quarterly,Q,Millions of Chained 2017 Dollars,Mil. of Chn. 2017 $,Seasonally Adjusted,SA,2023-09-29 07:50:06-05,2,Real Manufacturing and Trade Industries Sales ...
0,T10Y3M,2023-10-08,2023-10-08,10-Year Treasury Constant Maturity Minus 3-Mon...,1982-01-04,2023-10-06,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2023-10-06 16:02:10-05,95,Series is calculated as the spread between 10-...
0,T10Y2Y,2023-10-08,2023-10-08,10-Year Treasury Constant Maturity Minus 2-Yea...,1976-06-01,2023-10-06,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2023-10-06 16:02:04-05,100,"Starting with the update on June 21, 2019, the..."
0,BAMLH0A0HYM2,2023-10-08,2023-10-08,ICE BofA US High Yield Index Option-Adjusted S...,1996-12-31,2023-10-05,"Daily, Close",D,Percent,%,Not Seasonally Adjusted,NSA,2023-10-06 08:54:11-05,93,The ICE BofA Option-Adjusted Spreads (OASs) ar...
0,BAA10Y,2023-10-08,2023-10-08,Moody's Seasoned Baa Corporate Bond Yield Rela...,1986-01-02,2023-10-05,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2023-10-06 16:02:02-05,75,Series is calculated as the spread between Moo...


In [15]:
tickers_frequencies = dict(zip(infos['id'],infos['frequency'].replace('Daily, Close','Daily')))

In [16]:
tickers_dataset = dict(zip(series_codes,data))

In [17]:
import json

possible_targets = ['USREC']

# Create a dictionary to store the JSON structure
infos_dictionary = {}

# Iterate through each row in the DataFrame
for index, row in infos.iterrows():
    # Extract the key from the 'id'
    key = row['id']
    
    # Remove the 'id' from the row data
    row_data = row.drop('id').to_dict()
    
    row_data['ts'] =tickers_dataset[key]

    if key in possible_targets:
        row_data['modeling_variable_type'] = 'Target'
    else:
        row_data['modeling_variable_type'] = 'Attribute'

    # Add the key-value pair to the JSON dictionary
    infos_dictionary[key] = row_data

# Convert the JSON dictionary to a JSON string
#json_string = json.dumps(infos_dictionary, indent=4)

# Print or save the JSON string
#print(infos_dictionary)

In [35]:
infos_dictionary

{'USREC': {'realtime_start': '2023-10-08',
  'realtime_end': '2023-10-08',
  'title': 'NBER based Recession Indicators for the United States from the Period following the Peak through the Trough',
  'observation_start': '1854-12-01',
  'observation_end': '2023-09-01',
  'frequency': 'Monthly',
  'frequency_short': 'M',
  'units': '+1 or 0',
  'units_short': '+1 or 0',
  'seasonal_adjustment': 'Not Seasonally Adjusted',
  'seasonal_adjustment_short': 'NSA',
  'last_updated': '2023-10-02 18:01:01-05',
  'popularity': '69',
  'notes': "This time series is an interpretation of US Business Cycle Expansions and Contractions data provided by The National Bureau of Economic Research (http://www.nber.org/cycles/cyclesmain.html) (NBER). Our time series is composed of dummy variables that represent periods of expansion and recession. The NBER identifies months and quarters of turning points without designating a date within the period that turning points occurred. The dummy variable adopts an arb

In [30]:
x = pd.Series(['Y','Q',"M","D"],dtype='category')
x

(array([3, 2, 1, 0]), array(['D', 'M', 'Q', 'Y'], dtype=object))

In [32]:
category_frequencies = {
    "Y": 1,
    "Q": 2,
    "M": 3,
    "D":4
}


sorted_frequencies_categories = sorted(category_frequencies.keys(), key=lambda x: category_frequencies[x])
print(sorted_frequencies_categories)


['Y', 'Q', 'M', 'D']


In [34]:
infos[['id','frequency_short']]

,id,frequency_short
0,USREC,M
0,GDPC1,Q
0,PIECTR,Q
0,PRS85006013,Q
0,IPB50001SQ,Q
0,CQRMTSPL,Q
0,T10Y3M,D
0,T10Y2Y,D
0,BAMLH0A0HYM2,D
0,BAA10Y,D


In [25]:
infos[['id','frequency_short']]

In [37]:
infos.query(f"id == 'ATLSBUEGEP' ")['frequency_short'].unique()[0]

'M'

In [47]:
tickers_dataset['USREC'].index.min()

Timestamp('1854-12-01 00:00:00')

In [44]:
d = {"a":[1,2,3]}
if type(d) == dict:
    print("aaa")
else:
    print("fff")
    print(type(d))

aaa


In [32]:
from utils.extract_data import *
from utils.time_series_preprocessing import * 

def gathering_data_for_modeling(tickers_dataset,
                                infos_dataset,
                                chosen_target,
                                frequency_ajustment='higher_to_lower',
                                begin_date=None,
                                end_date=None,
                                include_seasonal_ajusted_series=True,
                                agg_method = "last",
                                **dict_kwargs):
    
    infos_dictionary = get_data_dictionary(tickers_dataset,infos_dataset,resample_agg='last',**dict_kwargs)

    category_frequencies = {"Y": 1,"Q": 2,"M": 3,"D":4}
    
    # Define a custom key function to map category labels to frequencies
    
    def custom_key(category):
        return category_frequencies.get(category, 0) 
    
    max_frequency = max(infos['frequency_short'], key=custom_key)
    min_frequeny =  min(infos['frequency_short'], key=custom_key)

    if frequency_ajustment == 'higher_to_lower':
        chosen_freq = max_frequency
    elif frequency_ajustment == 'lower_to_higher':
        chosen_freq = min_frequeny
    elif frequency_ajustment == 'target_frequency':
        chosen_freq = infos_dataset.query(f"if == '{chosen_target}' ")['frequency_short'].unique()[0]
    else:
        raise ValueError("Unsupported frequency ajustment. Choose from ''higher_to_lower' ,'lower_to_higher' ,'target_frequency'")


    target_series = infos_dictionary[chosen_target]['ts']

    series_resampled = []
    for k in infos_dictionary.keys():
        
        
        serie = k['ts']


        if begin_date is None:
            start = target_series.index().min()
        else:
            start = begin_date
        if end_date is None:
            end = target_series.index().max()
        else:
            end = end_date
        
        serie = serie[start:end]




        if type(agg_method) == str: 
            resampled_ts = resample_time_series(serie,desired_freq=chosen_freq,aggregation_method=agg_method)
        elif type(agg_method) == dict:
            for agg in agg_method.keys():
                if k in agg_method[agg]:
                    resampled_ts = resample_time_series(serie,desired_freq=chosen_freq,aggregation_method=agg)
                else:
                    pass

        series_resampled.append(resample_time_series)
        
        
        
        










    
    


aa
